## 432. All O`one Data Structure

Design a data structure to store the strings' count with the ability to return the strings with minimum and maximum counts.

Implement the AllOne class:

AllOne() Initializes the object of the data structure.
inc(String key) Increments the count of the string key by 1. If key does not exist in the data structure, insert it with count 1.
dec(String key) Decrements the count of the string key by 1. If the count of key is 0 after the decrement, remove it from the data structure. It is guaranteed that key exists in the data structure before the decrement.
getMaxKey() Returns one of the keys with the maximal count. If no element exists, return an empty string "".
getMinKey() Returns one of the keys with the minimum count. If no element exists, return an empty string "".
Note that each function must run in O(1) average time complexity.

## Solution:
To solve this problem, we aim to maintain a doubly-linked list where each node represents a unique count and stores the keys (strings) with that count. The nodes of the list are kept in sorted order, that is, in increasing order of the counts.

Here's the underlying intuition behind the data structure and its operations:

* Using a Doubly Linked List: This allows easy insertion and deletion of nodes which represent counts of keys. A node can quickly connect to its neighbors without having to shift elements, as would be necessary in an array.

* Maintaining a root Node: The root node acts as a sentinel that enables the list to be circular, which simplifies the edge cases for insertions and deletions at the beginning and end of the list.

* Storing Keys in Sets within Nodes: Each node contains a set of keys with the corresponding count for that node. Using a set ensures that we handle the keys efficiently when incrementing and decrementing their counts.

* Hashtable to Store References to Nodes: A hashtable (nodes dictionary in Python code) keeps track of the nodes associated with each key. This enables constant-time access to the nodes when incrementing or decrementing counts.

## Operations Explained:

Increment (inc): When a key's count is incremented:

If the key is new, it is either added to a new node with count 1 or to an existing node if it directly follows the root node with a count of 1.

If the key already exists in a node, we either create a new node with an incremented count right after the current node (if the incremented count does not match the next node's count) or we add the key to the set of the next node.

Decrement (dec): When a key's count is decremented:

If the count comes down to 0, the key is removed from the hashtable.

If the key's count after decrementing doesn't match the previous node's count (or no previous node exists), a new node is created with the decremented count. Otherwise, the key is added to the previous node's set.

Get Maximum (getMaxKey) and Get Minimum (getMinKey): To retrieve the keys with maximum and minimum counts, we can directly access the sets in the last and first nodes, respectively, from the root due to the order maintained in the doubly-linked list.

By carefully maintaining the structure of the linked list and the hashtable, we can achieve the required constant average time complexity for each operation mandated by the problem.



In [ ]:
class Node:
    def __init__(self, key='', count=0):
        self.prev = None
        self.next = None
        self.count = count  # The number of times this key appears
        self.keys = {key}  # A set of keys with the same count

    def insert(self, node):
        """Insert the given node after this node."""
        node.prev = self
        node.next = self.next
        self.next.prev = node
        self.next = node
        return node

    def remove(self):
        """Remove this node from the list."""
        self.prev.next = self.next
        self.next.prev = self.prev
        
class AllOne:
    def __init__(self):
        self.root = Node()  # Dummy root node of the doubly linked list
        self.root.next = self.root  # Initialize root to point to itself
        self.root.prev = self.root
        self.nodes = {}  # Key to node dictionary

    def inc(self, key: str) -> None:
        """Increase the count of the key by 1."""
        if key not in self.nodes:
            # Key not in list, so add with count 1
            if self.root.next == self.root or self.root.next.count > 1:
                # Insert new node after root if next node's count is more than 1
                self.nodes[key] = self.root.insert(Node(key, 1))
            else:
                # Otherwise, add the key to the next node's keys
                self.root.next.keys.add(key)
                self.nodes[key] = self.root.next
        else:
            current = self.nodes[key]
            next_node = current.next
            if next_node == self.root or next_node.count > current.count + 1:
                # Insert new node if there's no node for count = current.count + 1
                self.nodes[key] = current.insert(Node(key, current.count + 1))
            else:
                next_node.keys.add(key)
                self.nodes[key] = next_node
            current.keys.discard(key)
            if not current.keys:
                # Remove current node if there are no keys left in it
                current.remove()

    def dec(self, key: str) -> None:
        """Decrease the count of the key by 1."""
        current = self.nodes[key]
        if current.count == 1:
            # Remove the key if its count goes to 0
            del self.nodes[key]
        else:
            previous = current.prev
            if previous == self.root or previous.count < current.count - 1:
                # Insert new node if there's no node for count = current.count - 1
                self.nodes[key] = previous.insert(Node(key, current.count - 1))
            else:
                # Otherwise, add the key to the previous node's keys
                previous.keys.add(key)
                self.nodes[key] = previous
        current.keys.discard(key)
        if not current.keys:
            # Remove current node if there are no keys left in it
            current.remove()

    def getMaxKey(self) -> str:
        """Return a key with the highest count. If no keys are present, return an empty string."""
        if self.root.prev == self.root:
            return ""  # The list is empty
        return next(iter(self.root.prev.keys))

    def getMinKey(self) -> str:
        """Return a key with the lowest count. If no keys are present, return an empty string."""
        if self.root.next == self.root:
            return ""  # The list is empty
        return next(iter(self.root.next.keys))

# Your AllOne object will be instantiated and called as such:
# obj = AllOne()
# obj.inc(key)
# obj.dec(key)
# max_key = obj.getMaxKey()
# min_key = obj.getMinKey()